### Objective:
The objective of the notebook is to -
* Build the final model of Prophet algorithm using the best hyperparameter set identified using Backtesting and score the test set to get a performance metric
* For forecasting future periods, we will re-train the model with the same hyperparameter set on the train + validation + test set to capture the patterns in the test set and then forecast future N periods

In [0]:
import yaml
import inspect
import glob
import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
from distutils.command.config import config
from tqdm.auto import tqdm
from datetime import timedelta
from datetime import datetime
import mlflow
import dotsi
from sklearn.metrics import mean_absolute_error,mean_squared_error
import os
import logging
from prophet.utilities import regressor_coefficients

In [0]:
# logging part
p_dir = "/tmp/"
log_file = "Prophet_model_eval_retraining_scoring" + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+ ").log"

logger = logging.getLogger('custom_log')
logger.setLevel(logging.DEBUG)

# Applying necessary formatter
fh = logging.FileHandler(p_dir+log_file, mode = 'a')
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
fh.setFormatter(formatter)
logger.addHandler(fh)

In [0]:
# Getting the default settings of hyperparameters. Used to check that user-provided hyperparameters must always be a subset of these.
def get_default_args(func) -> dict:
    """Function to get the default values of the hyperparameters for the given algorithm

    Parameters
    ----------
    func : constructor of the respective algorithm
        The name of the algorithm (Eg: Prophet,SARIMAX)

    Returns
    -------
    dict
        returns a dictionary of hyperparameters and the corresponding default values for the given algorithm
    """
    
    signature = inspect.signature(func)
    return {
        k: v.default if v.default is not inspect.Parameter.empty else None
        for k, v in signature.parameters.items()
        if k != 'self'
    }
default_hpps = get_default_args(Prophet)

In [0]:
def broadcast_holidays(
    config_holidays: dict,
    year_list: list =[2018, 2019, 2020, 2021, 2022],
    country_name: str ="US",
    holiday_lower_window: int =7,
    holiday_upper_window: int =7,
) -> pd.DataFrame:
    """Function to return the dataframe of holidays for the given time period using Prophet's make_holidays_df()

    Parameters
    ----------
    config_holidays : dict
        the additional list of holidays and its respective dates provided by the user in config file
    year_list : list, optional
        the list of years for which we need the holidays , by default [2018, 2019, 2020, 2021, 2022]
    country_name : str, optional
        Name of the country based on which holidays can be decided, by default "US"
    holiday_lower_window : int, optional
        lower limit of the window, by default 7
    holiday_upper_window : int, optional
        upper limit of the window, by default 7

    Returns
    -------
    pd.DataFrame
        Returns a dataframe of holidays for the given time period
    """
    holidays = make_holidays_df(year_list, country_name)
    # Add window
    holidays['lower_window'] = -holiday_lower_window
    holidays['upper_window'] = holiday_upper_window
    
    # Adding additional holidays
    if config_holidays is not None:
        for ad_hol in config_holidays.keys():
            temp_df = pd.DataFrame({'holiday':ad_hol,
                                    'ds': pd.to_datetime(config_holidays[ad_hol]['ds']),
                                    'lower_window': -holiday_lower_window,
                                    'upper_window': holiday_upper_window})
            holidays = pd.concat([holidays,temp_df])
    
    # Dropping duplicates if exists any
    holidays = holidays.drop_duplicates().reset_index(drop = True)
    return holidays

In [0]:
%run ../../../0_Config.ipynb

In [0]:
logger.info("Config file read")
assert set(app_config["Algorithms"]["Prophet"]["Hyperparameters"].keys()).issubset(set(default_hpps.keys())),\
           'keys supplied by the user for the Prophet Algorithm under Hyperparameters must be valid'

# For exporting the config file
temp_config = app_config.copy()

In [0]:
def frange(start,stop,step= 1):
    l = []
    i = start
    while(i < stop):
        l.append(round(i,len(str(step))))
        i = i+step
    return l

def drange(hyperparameters):
    l=[]
    for key in hyperparameters.keys():
        val = hyperparameters[key]
        if 'range' in val:
            val = val.replace('range','frange')
            new_str = 'total_list = '  + val
            _locals = locals()
            exec(new_str,globals(),_locals)
            without_dup = list(set(_locals['total_list']))
            hyperparameters[key] = without_dup
    return hyperparameters

In [0]:
fit_ = app_config["Algorithms"]["Prophet"]["Hyperparameters"]
fit_new = {}
for key in fit_.keys():
    temp = []
    for val in fit_[key]:
        if(type(val) == list):
            val = str(val)
        if((val!='None') and (val!='Null') and (val!=None)):
            temp.append(val)
    if(len(temp)>0):
        fit_new[key] = temp
        
app_config["Algorithms"]["Prophet"]["Hyperparameters"] = fit_new

In [0]:
# Create the algo and logs directory for storing the results
output_directory = app_config['output_dir_path']
root_dir = "Modeling_Results"
algorithm = "Prophet"
algo_path = os.path.join(output_directory,root_dir,algorithm)
if not os.path.exists(algo_path):
    os.makedirs(algo_path)
logger.info("Created algorithm directory")    

logs_path = os.path.join(output_directory,root_dir,'logs',algorithm)
if not os.path.exists(logs_path):
    os.makedirs(logs_path)
    
logger.info("Created logs directory")

config_path = os.path.join(app_config['output_dir_path'],"Modeling_Results","config")
if not os.path.exists(config_path):
    os.makedirs(config_path)
    
logger.info("Created config directory")

In [0]:
hyperparameters_conf = dict(app_config["Algorithms"]["Prophet"]["Hyperparameters"])
# print(hyperparameters_conf)

modeling_granularity_conf = app_config["modeling_granularity"]
# print(modeling_granularity_conf)

# Rename Start date and DV config
dv_config = app_config["dependent_variable"]
ds_config = app_config["date_var"]

# pos and neg corr broadcast
corr_config = dict(app_config['Algorithms']['Prophet']['exogenous_variables'])
corr_config_broadcast = dotsi.Dict({"value":corr_config})

# Eval metric broadcast
broadcast_metric = dotsi.Dict({"value":app_config['validation']['metric']})
broadcast_test_periods = dotsi.Dict({"value":app_config["validation"]["no_of_test_periods"]})
broadcast_regressor_mode = dotsi.Dict({"value":app_config["Algorithms"]["Prophet"]["regressor_mode"]})
broadcast_tracking = dotsi.Dict({"value":app_config['tracking']})
mlflow_tracking_check = dotsi.Dict({"value":"Out of Sample"})
# ===================================================================================

# Broadcasting
if app_config["Algorithms"]["Prophet"]["Holidays"]["include_holidays"] == True:
    aa = app_config["Algorithms"]["Prophet"]["Holidays"]
    holidays_broadcast = broadcast_holidays(aa['additional_holidays'],aa['years'],aa['country'],aa['holiday_lower_window'],aa['holiday_upper_window'])
    holidays_broadcast = dotsi.Dict({"value":holidays_broadcast})
else:
    holidays_broadcast = dotsi.Dict({"value":None})
    
broadcast_regressors = dotsi.Dict({"value":list(set(corr_config['positive_corr']+corr_config['negative_corr']+corr_config['uncertain_corr']))})
broadcast_granularity =dotsi.Dict({"value":modeling_granularity_conf})
broadcast_hyper_parameters = dotsi.Dict({"value":hyperparameters_conf})
broadcast_agg_metrics_req = dotsi.Dict({"value":app_config["validation"]["agg_metrics_req"]})

logger.info("Broadcasted the required variables")

In [0]:
# Reading the latest file based on timestamp
all_files = [file for file in os.listdir(algo_path)]
best_hyp_files = [file for file in all_files if "Best_hyperparameters (" in file]
best_hyp_files = [file.replace(".csv","") for file in best_hyp_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in best_hyp_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in best_hyp_files if max_date in x]
best_hyp_param_results_file_path = os.path.join(algo_path,req_file_name[0]+".csv")
print(best_hyp_param_results_file_path)

best_hyperparam_results = pd.read_csv(best_hyp_param_results_file_path)
best_hyperparam_results = best_hyperparam_results[best_hyperparam_results['status']=='success'].reset_index(drop = True)
best_hyperparam_results[modeling_granularity_conf] = best_hyperparam_results[modeling_granularity_conf].astype(str)
best_hyperparam_results.replace(['true'],True, inplace = True)
best_hyperparam_results.replace(['false'],False, inplace = True)
best_hyperparam_results_broadcast = dotsi.Dict({"value":best_hyperparam_results})
logger.info("Read the best hyperparamter results")
best_hyperparam_results

/dbfs/mnt/solutionsadls_data/modeling_results_python/Modeling_Results/Prophet/Best_hyperparameters (2023-01-13-05-34-40).csv
Out[18]:

,Div_No,Store_No,Base_UPC,seasonality_prior_scale,changepoint_prior_scale,yearly_seasonality,mape,wmape,bias,tracking_signal,mae,rmse,status
0,24,8,2200015934,0.9,0.01,True,71.544410,42.661999,0.026536,-0.227906,2.490881,2.929188,success
1,24,14,4000042206,0.9,0.01,True,93.907670,73.514311,-2.813054,-0.025078,8.825539,9.877684,success
2,24,15,4000046410,0.9,0.01,True,147.603857,88.713018,-0.830174,-0.648747,6.231397,7.067459,success
3,24,17,4000000032,0.9,0.01,True,89.705656,63.424032,-1.080522,0.515436,5.768601,6.929459,success
4,24,53,4000000263,0.5,0.01,True,154.538461,98.742796,18.991775,3.870443,22.795517,24.429324,success
5,24,55,4000000051,0.9,0.01,True,193.146757,130.875273,0.309056,-0.552168,4.642972,6.145017,success
6,24,55,4000005851,0.5,0.01,True,371.128422,226.021793,-2.182556,1.244516,4.992277,5.922593,success


In [0]:
# Reading feature selected output and using the significant variables as idvs in modeling
feature_selection_info = app_config['Algorithms']['Prophet']['feature_selection']
broadcast_use_features = dotsi.Dict({"value":feature_selection_info['use_feature_selected_idvs']})
if(feature_selection_info['use_feature_selected_idvs']):
    if(feature_selection_info['approach']=='lasso_cvglmnet'):
        output_folder = app_config['output_dir_path']+"/Feature_Selection/Lasso/"
    # Reading the latest input file based on timestamp
    coeff_op_files = [file for file in os.listdir(output_folder)]
    coeff_op_files = [file.replace(".csv","") for file in coeff_op_files]
    version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in coeff_op_files]
    max_date = max(version_dates)
    max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
    req_file_name = [x for x in coeff_op_files if max_date in x]
    coeff_op_file_path = os.path.join(output_folder,req_file_name[0] + ".csv")
    print(coeff_op_file_path)

    # Reading the data
    coeff_df = pd.read_csv(coeff_op_file_path)
    coeff_df = coeff_df[coeff_df['status']=='success']
    # print(coeff_df.shape)
    coeff_df[modeling_granularity_conf] = coeff_df[modeling_granularity_conf].astype(str)
    idvs_len = len(feature_selection_info['must_have_idvs'])
    if(idvs_len>0):
        temp1 = coeff_df[modeling_granularity_conf].drop_duplicates()
        temp1['temp'] = 1
        temp2 = pd.DataFrame({'IDV':feature_selection_info['must_have_idvs']})
        temp2['temp'] = 1
        temp = temp1.join(temp2, on = 'temp', how ='left')
        req_cols = modeling_granularity_conf + ['IDV']
        coeff_df = coeff_df.drop_duplicates()
    coeffs_broadcast = dotsi.Dict({"value":coeff_df})
    broadcast_regressors = dotsi.Dict({"value":list(coeff_df['IDV'].unique())})
# display(coeff_df)

/dbfs/mnt/solutionsadls_data/modeling_results_python/Feature_Selection/Lasso/lasso_feature_selection_results (2023-01-13-04-31-21).csv

In [0]:
def get_forecast_UDF(df_data: pd.DataFrame)-> pd.DataFrame:
    """Function to perform final model building using the train data and score on the test data utilizing the broadcasted details from the config file

    Parameters
    ----------
    df_data : pd.DataFrame
        The dataset containing values for all the required variables

    Returns
    -------
    pd.DataFrame
      Returns a dataframe with the granularity,date,independent variables contributions if any and performance metrics for the training and the testing set
    """
    try:
        test_periods = int(broadcast_test_periods.value)
        if(broadcast_agg_metrics_req.value == True):
            train_index_start = df_data["train_index_start"].iloc[0]
            train_index_end = df_data["train_index_end"].iloc[0]
            test_i = df_data["test_index_end"].iloc[0]
            window_no = str(str(train_index_start)+" "+str(train_index_end)+" "+str(test_i)+" "+str(df_data["window_no"].iloc[0]))
        else:
            train_index_end = len(df_data) - test_periods
            test_i = len(df_data)
            window_no = str(1)
            
        df_data = df_data.sort_values(by=['ds'],ascending=True)
        hpt = best_hyperparam_results_broadcast.value
        regressor_mode = broadcast_regressor_mode.value

        # broadcast_granularity
        broadcast_gran = broadcast_granularity.value

        # get best hyperparameters for the given modeling granularity
        for x in list(broadcast_gran):
            hpt = hpt[hpt[x] == df_data[x].iloc[0]]

        # Train - test split
        train = df_data.iloc[:train_index_end]
        test = df_data.iloc[train_index_end:test_i]
        
        # Updating the default arguments with the parameters provided in the config
        hp_config = list(broadcast_hyper_parameters.value)
        def_args = get_default_args(Prophet)
        for x in hp_config:
            def_args[x] = hpt[x].iloc[0]
        if holidays_broadcast.value is not None:
            def_args["holidays"] = holidays_broadcast.value

        if(regressor_mode not in ['additive','multiplicative']):
            regressor_mode = def_args['seasonality_mode'] 
        # Calling the Prophet constructor with the hyperparameters of interest  
        m = Prophet(**def_args)
        
        if(broadcast_use_features.value==True):
            # Reading regressors from feature selection
            coeffs_df = coeffs_broadcast.value
            for x in broadcast_gran:
                coeffs_df = coeffs_df[coeffs_df[x] == df_data[x].iloc[0]]
            regressors = list(coeffs_df['IDV'].values)
            temp_list = list(set(broadcast_regressors.value) - set(regressors))
        else:
            # Appending regressors based on the sign of correlation
            corr_var = corr_config_broadcast.value
            regressors = list(set(corr_var["positive_corr"] + corr_var["negative_corr"]+corr_var['uncertain_corr']))

            temp_list1 = []
            # Removing regressors based on the correlation
            if(corr_var["consider_correlation"]):   
                for x in corr_var["positive_corr"]:
                    if(train[['y',x]].corr().iloc[0][1]<0):
                        temp_list1.append(x)
                for x in corr_var["negative_corr"]:
                    if (x not in temp_list1):
                        if(train[['y',x]].corr().iloc[0][1]>0):
                            temp_list1.append(x)   
                regressors = list(set(regressors) - set(temp_list1))
                
            # Checking for variance in the regressor
            temp_list2 = []
            if len(regressors)>0:
                for ex_var in regressors:  
                    mean = train[ex_var].mean()
                    std = train[ex_var].std()
                    if mean == 0:
                        if std <= 0.001:
                            temp_list2.append(ex_var)
                    else:
                        if abs(std/mean) <= 0.01:
                            temp_list2.append(ex_var)
                            
            regressors = list(set(regressors) - set(temp_list2)) 
            temp_list = temp_list1+temp_list2
            
        for var in regressors:
            m.add_regressor(var,mode = regressor_mode)
        m.fit(train)

        test_res = m.predict(test)
        test_res[['test_flag','test_flag_agg']] = 1
        train_res = m.predict(train)
        train_res['test_flag'] = np.where(train_res['ds']<df_data.iloc[-test_periods:]['ds'].min(),0,1)
        train_res['test_flag_agg'] = 0
        forecast_pd = pd.concat([train_res,test_res],ignore_index = True)
        
        seasonal_cols = ['yearly','weekly','daily']
        for var in seasonal_cols:
            if(var not in forecast_pd.columns):
                forecast_pd[var] = 0
        if holidays_broadcast.value is not None:
            holidays_list = list(m.train_holiday_names.values) + ['holidays']
        else:
            holidays_list = []
            temp_list.append("holidays")
        results_pd = forecast_pd[['ds', 'yhat', 'yhat_upper','yhat_lower','trend','test_flag','test_flag_agg']\
                               +regressors+seasonal_cols+holidays_list]

        # Contribution Calculation
        if(m.seasonality_mode == 'multiplicative'):
            for var in ['yearly','weekly','daily']:
                if(var in results_pd.columns):
                    results_pd[var] = results_pd[var]*results_pd['trend']
            for var in holidays_list :
                results_pd[var] = results_pd[var]*results_pd['trend']

        if(len(regressors)>0):
            reg_coeff = regressor_coefficients(m)
            if(reg_coeff['regressor_mode'].unique()[0]=='multiplicative'):
                for var in regressors:
                    results_pd[var] = results_pd[var]*results_pd['trend']

        results_pd = pd.merge(results_pd, df_data[['y','ds']+broadcast_gran], how = "left",on = "ds")
        # Sales or Quantity can't be negative hence
        results_pd["yhat"] = np.where(results_pd["yhat"]<0,0,results_pd["yhat"])
        results_pd["yhat_upper"] = np.where(results_pd["yhat_upper"]<0,0,results_pd["yhat_upper"])
        results_pd["yhat_lower"] = np.where(results_pd["yhat_lower"]<0,0,results_pd["yhat_lower"])

        # to handle erroneous results epsilon is set to 1.
        epsilon = 1

        temp_data1 = pd.DataFrame(index= range(1))
        temp_data2 = pd.DataFrame()
        results_pd_temp = results_pd[~((results_pd['test_flag']==1) & (results_pd['test_flag_agg']==0))]
        for val in [1,0]:
            temp_data = results_pd_temp[results_pd_temp['test_flag_agg']==val]
            y_pred = temp_data['yhat']
            y_true = temp_data['y']

            temp_data1['test_flag_agg'] = val
            # Eval. metrics calculation
            temp_data1['mape'] = np.mean(np.abs(y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))*100  
            temp_data1['wmape'] = np.sum(np.abs(y_true - y_pred)) / np.maximum(np.sum(np.abs(y_true)),epsilon)*100  
            temp_data1['bias'] = np.mean((y_true - y_pred))  
            temp_data1['tracking_signal'] = np.sum((y_true - y_pred)) / np.mean(np.abs(y_true - y_pred))
            temp_data1['mae'] = mean_absolute_error(y_true, y_pred)
            temp_data1['rmse']=np.sqrt(mean_squared_error(y_true, y_pred))
            temp_data2 = pd.concat([temp_data2,temp_data1],ignore_index = True)
        
        results_pd = pd.merge(results_pd,temp_data2,how='left',on='test_flag_agg')
        
        # To adhere to defined schema
        for x in broadcast_gran:   
            results_pd[x] = results_pd[x].astype(str)

        # Append Hyperparameters used
        for x in hp_config:
            results_pd[x] = hpt[x].iloc[0]

        if(len(temp_list)>0):
            for var in temp_list:
                results_pd[var] = 0
      
        results_pd.loc[results_pd['yhat']==0,['trend']+regressors+seasonal_cols+holidays_list] = 0
        results_pd["window"] = window_no

        # Get the experiment id
        tracking_value = broadcast_tracking.value.copy()
            
        if(mlflow_tracking_check.value == "Out of Sample" and tracking_value["tracking_needed"] == True):

            if(tracking_value['type']!="Managed"):
                if(tracking_value['tracking_uri'] is not None):
                    mlflow.set_tracking_uri("file:"+tracking_value['tracking_uri'])
                    experiment_id = mlflow.set_experiment(tracking_value["mlflow_experiment_id"])
                    tracking_value['mlflow_experiment_id'] = experiment_id.experiment_id

            #Add MLFlow code here
            with mlflow.start_run(experiment_id = tracking_value['mlflow_experiment_id']):
                mlflow.log_param('algorithm', 'Prophet')
                mlflow.log_param('result_type', 'out_of_sample')
                for x in broadcast_gran:
                    mlflow.log_param(x, results_pd[x].iloc[0])
                for x in hp_config:
                    mlflow.log_param(x, results_pd[x].iloc[0])
                test = results_pd[results_pd['test_flag']==1].reset_index(drop = True)
                for x in ["mape","wmape","bias","tracking_signal","mae","rmse"]:
                    mlflow.log_metric(x, test[x].iloc[0])
                    
        results_pd['status'] = 'success'
        return results_pd
    except Exception as e:
        if holidays_broadcast.value is not None:
            holidays_list = list(holidays_broadcast.value['holiday'].unique()) + ['holidays']
        else:
            holidays_list = ['holidays']
        results_pd = pd.DataFrame(columns = [['ds', 'y', 'yhat','yhat_upper','yhat_lower','mape','wmape','bias','tracking_signal','mae','rmse']+\
                                             list(broadcast_hyper_parameters.value.keys()) + ['status','test_flag','test_flag_agg','window'] + broadcast_granularity.value+\
                                             broadcast_regressors.value + ['trend','yearly','weekly','daily'] +\
                                            holidays_list],index = range(1))
        results_pd[broadcast_granularity.value] = df_data[broadcast_granularity.value].head(1).reset_index(drop = True)
        for x in broadcast_granularity.value:
              results_pd[x] = results_pd[x].astype(str)
        results_pd['status'] = str(e)
        
        return results_pd

#### Loading the latest Missing_value_treatment file
##### Please update the reading path with the required data path if "Missing value treatment" was not run

In [0]:
# Reading the latest input file based on timestamp
all_files = [file for file in os.listdir(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment")]
missing_op_files = [file for file in all_files if "Missing_value_treatment_results (" in file]
missing_op_files = [file.replace(".csv","") for file in missing_op_files]
version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in missing_op_files]
max_date = max(version_dates)
max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
req_file_name = [x for x in missing_op_files if max_date in x]
missing_op_file_path = os.path.join(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment",req_file_name[0]+'.csv')
# print(missing_op_file_path)

# Reading the data
df = pd.read_csv(missing_op_file_path)
# print(df.shape)

df.rename(columns = {ds_config:"ds", dv_config:"y"}, inplace = True)
df['ds'] = pd.to_datetime(df['ds'])
df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)

logger.info("Data loaded")
# print(list(broadcast_hyper_parameters.value.keys()))

gbcp = list(modeling_granularity_conf)

In [0]:
if(app_config["validation"]["agg_metrics_req"]):

    # Creating windows and then calling the modeling function
    test_periods = int(broadcast_test_periods.value)
    window_test_periods = app_config["validation"]["agg_metrics_test_periods"]
    stride = app_config["validation"]["agg_metrics_stride"]

    # Getting the total number of weeks for each time series
    temp_df = df.groupby(modeling_granularity_conf).agg({'ds':'count'}).rename(columns={'ds': '#total_weeks'}).reset_index()
    df = df.merge(temp_df, on = modeling_granularity_conf ,how = "left")

    unique_skuXds = df[modeling_granularity_conf+["#total_weeks"]].drop_duplicates().reset_index(drop = True)

    final_list = []
    gran_len = len(modeling_granularity_conf)
    
    for row1 in range(0,len(unique_skuXds)): 
        Total_weeks = unique_skuXds.loc[row1,'#total_weeks']
        train_interval = int(Total_weeks-test_periods)
        j = 0
        for train_i in range(train_interval,Total_weeks,stride):
            if(train_i+window_test_periods <=Total_weeks):
                test_i = train_i+window_test_periods
                final_list.append([unique_skuXds.iloc[row1,index] for index in range(gran_len)] + [0,train_i,train_i+window_test_periods,j+1])
                j += 1

    # create all windows combination.
    df_windows = pd.DataFrame([tuple(x) for x in final_list],columns =modeling_granularity_conf+['train_index_start','train_index_end','test_index_end','window_no'])
    f_df = df.merge(df_windows,on=modeling_granularity_conf,how="left")
        
    f_df['gran_tempp'] = f_df[gbcp+["window_no"]].astype(str).sum(axis=1)
    unique_pdts = f_df['gran_tempp'].unique()
    new_results = pd.DataFrame()
    for pdt in unique_pdts:
        new_results = pd.concat([new_results,get_forecast_UDF(f_df[f_df['gran_tempp']==pdt])])
            
    new_results.to_csv(algo_path+"/Out_of_sample_results_window_level ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
    logger.info("Completed Backtesting")
    
    # Reading the latest Out_of_sample_results_window_level file based on timestamp
    all_files = [file for file in os.listdir(algo_path)]
    backtesting_files = [file for file in all_files if "Out_of_sample_results_window_level (" in file]
    backtesting_files = [file.replace(".csv","") for file in backtesting_files]
    version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in backtesting_files]
    max_date = max(version_dates)
    max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
    req_file_name = [x for x in backtesting_files if max_date in x]
    backtesting_results_file_path = os.path.join(algo_path,req_file_name[0] + ".csv")
    print(backtesting_results_file_path)

    # Reading the results of backtesting
    df = pd.read_csv(backtesting_results_file_path)
    df = df[df["status"] == "success"]
    
    df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)
    df['ds'] = pd.to_datetime(df['ds'])

    # Roll up the data at Modeling granularity window level
    df_hyperparameters = best_hyperparam_results[gbcp + list(hyperparameters_conf)]

    # performance metrics
    per_met = ['status',"test_flag_agg","window","mape","wmape","bias","tracking_signal","mae","rmse"]
    df_metrics = df[gbcp + per_met].drop_duplicates()
    df_metrics1 = df_metrics.groupby(gbcp + ['test_flag_agg','status'])[["mape","wmape","bias","tracking_signal","mae","rmse"]].mean().reset_index()

    # Remaining columns
    rem_cols = list(set(df.columns) - set(per_met+list(hyperparameters_conf))) + ['test_flag_agg']
    dot_cols = [col for col in df.columns if "." in col] #to handle "."s
    for col in dot_cols:
        df.rename(columns = {col:col.replace(".","dot")}, inplace = True)
        rem_cols[rem_cols.index(col)] = col.replace(".","dot")
    rem_df = df[rem_cols]
    
    # Removing the training dates which falls in the test period
    rem_df = rem_df[~((rem_df['test_flag']==1) & (rem_df['test_flag_agg']==0))]
    group_cols = gbcp + ['ds','test_flag','test_flag_agg']
    agg_cols = list(set(rem_cols) - set(group_cols))
    exprs = {x: "mean" for x in agg_cols}
    rem_df1 = rem_df.groupby(group_cols).agg(exprs).reset_index()
    temp_cols = [col[:-1] if 'avg(' in col else col for col in rem_df1.columns ]
    temp_cols = [col.replace('avg(','') for col in temp_cols]
    rem_df1.columns = temp_cols

    for col in dot_cols:
        rem_df1.rename(columns = {col.replace(".","dot"):col.replace("dot",".")}, inplace = True)
                            
    # combining all the data
    df_forecast = rem_df1.merge(df_metrics1, on = gbcp + ['test_flag_agg'], how='left')
    df_forecast = df_forecast.merge(df_hyperparameters, on = gbcp , how='left')
    
else:    
    df['gran_tempp'] = df[gbcp].astype(str).sum(axis=1)
    unique_pdts = df['gran_tempp'].unique()
    df_forecast = pd.DataFrame()
    for pdt in unique_pdts:
        df_forecast = pd.concat([df_forecast,get_forecast_UDF(df[df['gran_tempp']==pdt])])
            
del(df_forecast['test_flag_agg'])
df_forecast['algorithm'] = 'Prophet'

11:15:06 - cmdstanpy - INFO - Chain [1] start processing
11:15:06 - cmdstanpy - INFO - Chain [1] done processing
11:15:11 - cmdstanpy - INFO - Chain [1] start processing
11:15:11 - cmdstanpy - INFO - Chain [1] done processing
11:15:18 - cmdstanpy - INFO - Chain [1] start processing
11:15:18 - cmdstanpy - INFO - Chain [1] done processing
11:15:23 - cmdstanpy - INFO - Chain [1] start processing
11:15:23 - cmdstanpy - INFO - Chain [1] done processing
11:15:29 - cmdstanpy - INFO - Chain [1] start processing
11:15:29 - cmdstanpy - INFO - Chain [1] done processing
11:15:34 - cmdstanpy - INFO - Chain [1] start processing
11:15:34 - cmdstanpy - INFO - Chain [1] done processing
11:15:40 - cmdstanpy - INFO - Chain [1] start processing
11:15:43 - cmdstanpy - INFO - Chain [1] done processing

In [0]:
df_forecast.to_csv(algo_path+"/Out_of_sample_evaluation_results ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
logger.info("Exported Out of sample evaluation results")

### Predicting future timeperiods
The following code assumes that the X-variables for the required future time periods are available for each modeling granularity

Uncomment the below cells if wants to predict the future, update the df respectively such that it contains entire historical data as well as idvs data for the required future forecast time periods

In [0]:
# broadcast_test_periods =  broadcast_variable_conf(4) # Provide the no. of timeperiods to forecast in the future

In [0]:
## Reading the latest input file based on timestamp
# all_files = [file for file in os.listdir(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment")]
# missing_op_files = [file for file in all_files if "Missing_value_treatment_results (" in file]
# missing_op_files = [file.replace(".csv","") for file in missing_op_files]
# version_dates = [datetime.strptime(x.split('(')[1].replace(')',''), '%Y-%m-%d-%H-%M-%S') for x in missing_op_files]
# max_date = max(version_dates)
# max_date = max_date.strftime('%Y-%m-%d-%H-%M-%S')
# req_file_name = [x for x in missing_op_files if max_date in x]
# missing_op_file_path = os.path.join(app_config['output_dir_path']+"/Data_Processing/Missing_value_treatment",req_file_name[0]+'.csv')
## print(missing_op_file_path)

## Reading the data
# df = pd.read_csv(missing_op_file_path)
## print(df.shape)

# df.rename(columns = {ds_config:"ds", dv_config:"y"}, inplace = True)
# df['ds'] = pd.to_datetime(df['ds'])
# df[modeling_granularity_conf] = df[modeling_granularity_conf].astype(str)

# # Broadcasting again with the "Future forecast" value since we won't be tracking the future forecast results
# mlflow_tracking_check = broadcast_required_info("Future forecast")
# logger.info("Data which contains the future forecast periods is loaded")

# gbcp = list(modeling_granularity_conf)

In [0]:
# df['gran_tempp'] = df[gbcp].astype(str).sum(axis=1)
# unique_pdts = df['gran_tempp'].unique()
# df_forecast = pd.DataFrame()
# for pdt in unique_pdts:
#     df_forecast = pd.concat([df_forecast,get_forecast_UDF(df[df['gran_tempp']==pdt])])
            
# del(df_forecast['test_flag_agg'])
# df_forecast['algorithm'] = 'Prophet'

In [0]:
# df_forecast.to_csv(algo_path + "/Future_forecast_results ("+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+").csv", index = False)
# logger.info("Exported future forecast results")

In [0]:
# Exporting config file
config_file_name = "config_for_exp_id_"+str(broadcast_tracking.value['mlflow_experiment_id']) + " (" +datetime.today().strftime('%Y-%m-%d-%H-%M-%S-%f')[:-3]+").yml"
config_path1 = os.path.join(config_path,config_file_name)
with open(config_path1, 'w') as file:
    yaml.dump(temp_config, file, default_flow_style=False,sort_keys=False)

In [0]:
# Move from tmp directory to req. location in datalake
import platform
plat_sys = platform.system()

if(plat_sys!='Windows'):
    log_file = log_file.replace(' (', '\ \(').replace(')','\)')
    os.system('mv /tmp/{0} {1}'.format(log_file,logs_path))